In [ ]:
import numpy as np
from numba import njit


@njit(cache=True, fastmath=True)
def comb(n, k):
    """Implement comb(n,k) mathematical function."""
    if k < 0 or k > n:
        return 0
    if k > n - k:
        k = n - k  # Take advantage of symmetry
    c = 1
    for i in range(k):
        c = c * (n - i) // (i + 1)
    return c


@njit(cache=True, fastmath=True)
def undifference(diff, initial_values):
    """
    Reconstruct original time series from an n-th order differenced series.

    Parameters
    ----------
    diff : array-like
        n-th order differenced series of length N - n
    initial_values : array-like
        The first n values of the original series before differencing (length n)

    Returns
    -------
    original : np.ndarray
        Reconstructed original series of length N
    """
    n = len(initial_values)
    kernel = np.array(
        [(-1) ** (k + 1) * comb(n, k) for k in range(1, n + 1)],
        dtype=initial_values.dtype,
    )
    original = np.empty((n + len(diff)), dtype=initial_values.dtype)
    original[:n] = initial_values

    for i, d in enumerate(diff):
        original[n + i] = np.dot(kernel, original[i : n + i][::-1]) + d

    return original


x = np.array([10, 13, 15, 14, 20, 100, 33, 5], dtype=np.float64)
d2 = x  # np.diff(x, n=3)
print(d2)
x_recovered = undifference(d2, x[:0])
print(x_recovered)
print(np.allclose(x, x_recovered))  # True

[ 10.  13.  15.  14.  20. 100.  33.   5.]
[ 10.  13.  15.  14.  20. 100.  33.   5.]
True


In [ ]:
import time

import numpy as np

from aeon.forecasting.stats import ARIMAForecaster

aeon_model = ARIMAForecaster(p=1, d=0, q=1, use_constant=True)
with open("temp.txt", "r") as f:
    model = np.fromfile(f, dtype=np.float64, count=750, sep=",")
aeon_model.fit(np.random.normal(loc=0, scale=1, size=750))
forecast = aeon_model.forecast_
t0 = time.time()
for _ in range(1000):
    aeon_model = ARIMAForecaster(p=1, d=0, q=1, use_constant=True)
    aeon_model.fit(model)
    forecast = aeon_model.forecast_
t1 = time.time()
time_per_cycle = (t1 - t0) / 100
print(f"Time taken for ARIMA model: {time_per_cycle} seconds")

Time taken for ARIMA model: 0.011441848278045653 seconds


In [ ]:
from math import comb as math_comb

from numba import njit

for n in range(10):
    for k in range(n - 1):
        assert comb(n, k) == math_comb(n, k)

In [11]:
import numpy as np

from aeon.transformations.series import DifferenceTransformer

X = np.array([[10, 13, 15, 14, 20, 100, 33, 5], [15, 56, 3, 104, 99, 100, 44, 51]])
transformer = DifferenceTransformer(order=1)
Xt = transformer.fit_transform(X)
print(Xt)
X_hat = transformer.inverse_transform(Xt, X)
print(X_hat)
assert np.allclose(X, X_hat)

[[  3   2  -1   6  80 -67 -28]
 [ 41 -53 101  -5   1 -56   7]]
[[ 10.  13.  15.  14.  20. 100.  33.   5.]
 [ 15.  56.   3. 104.  99. 100.  44.  51.]]


In [ ]:
import numpy as np

from aeon.forecasting.stats import ARIMA

y = np.array(
    [112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118], dtype=np.float64
)
model = ARIMA(p=1, d=1, q=1)
horizon = 3
preds = model.iterative_forecast(y, prediction_horizon=horizon)
assert preds.shape == (horizon,)
assert np.all(np.isfinite(preds))
model = ARIMA(p=1, d=0, q=1, use_constant=True)
preds = model.iterative_forecast(y, prediction_horizon=horizon)
print(preds.shape)
assert preds.shape == (horizon,)
print(f"ARIMA Forecast: {preds}")

C:\Users\alexb\Documents\University\PhD\aeon\aeon\aeon\forecasting\utils\_undifference.py:62: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'C', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  original[n + i] = np.dot(kernel, original[i : n + i][::-1]) + d


(3,)
ARIMA Forecast: [122.68454429 124.95567095 126.23839578]


In [2]:
from aeon.datasets import load_airline
from aeon.forecasting.stats import AutoARIMA

y = load_airline()
forecaster = AutoARIMA()
forecaster.forecast(y)

C:\Users\alexb\Documents\University\PhD\aeon\aeon\aeon\forecasting\utils\_undifference.py:62: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'C', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  original[n + i] = np.dot(kernel, original[i : n + i][::-1]) + d


482.1873214419662

In [1]:
from aeon.datasets import load_airline
from aeon.forecasting.stats import AutoETS

y = load_airline()
forecaster = AutoETS()
forecaster.forecast(y)

435.9312382780535